#### Currently you can monitor the feed at this address:

http://10.0.0.30:7777


In [1]:
#### Alpha - Mark 13
# Bolt Logic

#### Alpha - Mark 14
# Turing Check

#### Alpha - Mark 15
# Security

#### Beta - Mark 1
# Front End

#### Beta - Mark 2
# Accuracy


In [2]:
#!/usr/bin/env python3
import io, os, time, glob, shutil
from PIL import Image
import numpy as np
from time import sleep


class motionCapture():
    # A Motion detector and capturer
    def __init__(self):
        self.count = 0
        self.imgNum = 0
        self.notTimerSet = time.perf_counter()
        self.boltTimerSet = time.perf_counter()
        self.boltTimerComp = time.perf_counter()
        self.boltFlag = False
        
    def processFrame(self, frame):
        
        self.count += 1
        self.visitorID = ""
        timeStamp = time.strftime("%m%d%y_%H%M%S", time.gmtime())
        
        if (!self.boltFlag) and self.boltTimerComp-self.boltTimerSet >= userSettings.boltDelay
            self.bolt.engage()
            self.boltFlag = True
            self.boltTimerSet = self.boltTimerComp
            self.bolt.destroy()
                    
        if self.count%userSettings.captureRate==0:
            if self.count==userSettings.captureRate:
                self.referenceImg1 = np.array(Image.open(io.BytesIO(frame)))
                self.referenceImg2 = np.array(Image.open(io.BytesIO(frame)))
            else:
                self.testImg = np.array(Image.open(io.BytesIO(frame)))
                imgMSE1 = int(self.mse(self.referenceImg1, self.testImg))
                imgMSE2 = int(self.mse(self.referenceImg2, self.testImg))
                self.referenceImg2 = self.referenceImg1
                self.referenceImg1 = self.testImg
                
                
                    
                if (imgMSE1>=1000) or (imgMSE2>=1000):
                    noteMan.motLog(timeStamp)
                    if userSettings.inferIdentity:
                        noteMan.turingLog()
                        if True: #turingCore.inference(frame) == 1:)
                            noteMan.turingValLog()
                            self.visitorInference = idCore.inference(frame)
                            noteMan.idResLog(timeStamp, str(idCore.predicted_label), 
                                             str(self.visitorInference), str(idCore.prediction_strength))
#                          else:
#                             noteMan.turingFailLog()    
                            if idCore.predicted_label != 0:
                                if userSettings.boltControl:
                                    noteMan.boltLog()
                                    self.bolt = boltDriver(17)
                                    self.boltTimerComp = time.perf_counter()
                                    self.boltFlag = False

                    

                    if userSettings.captureStillOn==True:
                        if self.imgNum>=userSettings.extractionThreshold:
                            self.extractFiles(userSettings.storageDirectory, userSettings.captureDirectory, userSettings.extractionDirectory)
                            self.imgNum = 0
                        with open(userSettings.storageDirectory+userSettings.captureDirectory+"/" + timeStamp + ".jpg", "wb") as img:
                            img.write(frame)
                            noteMan.captLog(self.imgNum,timeStamp)
                            self.imgNum  += 1

                    if userSettings.notificationType==1 or userSettings.notificationType==2:
                        self.notTimerComp = time.perf_counter()
                        if self.notTimerComp-self.notTimerSet >= userSettings.notDelay:
                            print("Sending Notification")
                            noteMan.notifyOwner(self.visitorID)
                            self.notTimerSet = self.notTimerComp
                    noteMan.writeLog()


    def mse(self, imageA, imageB):
        # Mean Squared Error will determine change
        err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
        err /= float(imageA.shape[0] * imageA.shape[1])
        return err

    def extractFiles(self, storagePath, sampleFile, subFolder=''):

        # Setup path variables
        copyName = 'Extractable_'+(time.strftime("%m%d%y_%H%M%S", time.gmtime()))
        copyPath = storagePath+copyName
        zipPath = storagePath+subFolder+copyName
        transferredFiles = glob.glob(storagePath+sampleFile+'/*')
        
        # Copy files from src to destination
        shutil.copytree((storagePath+sampleFile), copyPath)

        # Zip temporary files into target location
        try:
            shutil.make_archive(zipPath, 'zip', storagePath, copyName)
        except OSError:
            pass

        # Clean up transferred Images
        for f in transferredFiles:
            os.remove(f)

        # Clean up tempory file
        shutil.rmtree(copyPath)

        return
    

In [3]:
#!/usr/bin/env python3
import io
import numpy as np
from PIL import Image
from IPython.display import clear_output
# import tensorflow as tf
# import tflite_runtime.interpreter as tf
import tflite_runtime.interpreter as tflite              
                  
class modelFrame():
    def __init__(self, neuralModel, labelActivation):
        self.conThres = labelActivation
        # Load TFLite model and allocate tensors (choose based on tflow runtime import)
#         self.interpreter = tf.lite.Interpreter(model_path=neuralModel)
        self.interpreter = tflite.Interpreter(model_path=neuralModel)        
        self.interpreter.allocate_tensors()

        # Get input and output tensors.
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
        
        log = "\n\nLoading . . .\n"
        log += "Core Loaded: "+ str(self.output_details) + "\n"
        log += str(self.output_details[0]['shape'][1])+" neural outputs, and "+str(len(userSettings.authGuest))+" in authorization List \n"
        
        if self.output_details[0]['shape'][1] == len(userSettings.authGuest):
            self.class_labels = userSettings.authGuest
            log += "Correlating Multiclass Model containing "+ str(len(self.class_labels))+" classes.\n"
            
        elif self.output_details[0]['shape'][1] == 2:
            self.class_labels = ['False','True']
            log += "Boolean Model.\n"
            
        else:
            log += "Failed! Unable to determine class labels!\n"
        
        noteMan.loadModelLog(log)
        return

    def inference(self, sample):
        self.sample = np.array(Image.open(io.BytesIO(sample)).resize((224,224),))
        self.sample = self.sample / 255.0
        self.cleanSample = np.expand_dims(self.sample, axis=0)
#         self.input_shape = self.input_details[0]['shape']
        input_data = np.array(self.cleanSample, dtype=np.float32)
        self.interpreter.set_tensor(self.input_details[0]['index'], input_data)
        self.interpreter.invoke()
        self.result_array = self.interpreter.get_tensor(self.output_details[0]['index'])
        self.predicted_index = np.argmax(self.result_array, axis=-1)
        self.prediction_strength = self.result_array[0][self.predicted_index]
        if self.prediction_strength >= self.conThres:
            self.predicted_label = self.class_labels[int(self.predicted_index)]
        else:
            self.predicted_label = 0
        return self.predicted_index


In [4]:
#!/usr/bin/env python3
import RPi.GPIO as GPIO
import time
from time import sleep

class boltDriver():

    def __init__(self, pinAssigned):
        # A Driver for LockCam Servo Motors
        self.servoPIN = pinAssigned
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.servoPIN, GPIO.OUT)
        self.p = GPIO.PWM(self.servoPIN, 50) # GPIO 17 for PWM with 50Hz
        self.p.start(7.4) # Initialization
        self.boltFlag = 0

    def engage(self):
        if self.boltFlag == 0:
            try:
                self.p.ChangeDutyCycle(12)
                self.boltFlag = 1
                time.sleep(0.5)
            except KeyboardInterrupt:
                self.p.stop()
                GPIO.cleanup()
        return

    def disengage(self):
        if self.boltFlag == 1:
            try:
                self.p.ChangeDutyCycle(7.4)
                self.boltFlag = 0
                time.sleep(0.5)
            except KeyboardInterrupt:
                self.p.stop()
                GPIO.cleanup()
        return

    def destroy(self):
        self.p.stop()
        GPIO.cleanup()
        self.p = 0
        return

In [5]:
#!/usr/bin/env python3
import smtplib, ssl


class notificationManager(): 
         
    def __init__(self):
        self.logMessage = ''
        self.emailMessage = ''
    def notifyOwner(self, notificationContent=""):
        port = userSettings.getEmailPort()
        smtpServer = userSettings.getSmtpServer()
        senderEmail = userSettings.getSenderEmail()
        receiverEmail = userSettings.getOwnerEmail()
        password = userSettings.getNotificationAuth()
        #password = input("Type your password and press enter:")
        self.emailMessage += """\
        Subject: LockCam Notification

        """
        if userSettings.notificationType == 1 or userSettings.notificationType() == 2: # better solve?
            self.emailMessage += """\
            Motion has been detected.

            """
            if notificationContent!='':
                self.emailMessage += """\
                The subject has been identified as """ + notificationContent
            else:
                self.emailMessage += """\
                The subject cannot be identified. """ + notificationContent

        context = ssl.create_default_context()
        with smtplib.SMTP(smtpServer, port) as server:
            server.ehlo()  # Can be omitted
            server.starttls(context=context)
            server.ehlo()  # Can be omitted
            server.login(senderEmail, password)
            server.sendmail(senderEmail, receiverEmail, self.emailMessage)
            self.emailMessage = ''
             
    def writeLog(self):
        self.logMessage += "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n"
        with open(userSettings.storageDirectory+userSettings.captureDirectory+"/"+"zLogs.txt","a") as l:
            l.write(self.logMessage+"\n \n")
            l.close()
        print(self.logMessage)
        self.logMessage = ''
            
    def motLog(self, stamp):
        self.logMessage += "\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n"
        self.logMessage += "Motion Detected at "+stamp+"\n"
        
    def turingLog(self):
        self.logMessage += "-----------------------------------\n"
        self.logMessage += "Running Turing Check . . . . . . . \n"
        
    def turingValLog(self):
        self.logMessage += " . . . . . . . . Humanity Validated\n"
        self.logMessage += "-----------------------------------\n"
        self.logMessage += "Running Identification . . . . . . \n"
        
    def turingFailLog(self):    
        self.logMessage += " . . . . non-Human Subject Detected/n"
        
    def idResLog(self, stamp, label, inference, strength):
        self.logMessage += " . . . . .ID Results: " + stamp + "\n"
        self.logMessage += "Predicted Label: "+ label +"\n"
        self.logMessage += "Inference: "+ inference +"\n"
        self.logMessage += "Prediction Strength: "+ strength +"\n"
        self.logMessage += "-----------------------------------\n"
        
    def boltLog(self):
        self.logMessage += "********* Disengaging Bolt ********\n"
        
    def captLog(self, imgNum, stamp):
        self.logMessage += "Frame "+str(imgNum)+" Captured at " + stamp + "\n"
    
    def loadModelLog(self, log):
        self.logMessage += log

In [6]:
#!/usr/bin/env python3
import yaml
    
class settingManager():
    def __init__(self):
        with open(r'setup/settings.yaml') as file:   ### This needs to be made CWD ####
            self.userSettingsDict = yaml.load(file, Loader=yaml.FullLoader)
            
        with open(r'setup/authorizations.yaml') as file:   ### This needs to be made CWD ####
            self.authorizationsDict = yaml.load(file, Loader=yaml.FullLoader)
            
        # Resource Settings
        self.storageDirectory = self.userSettingsDict['storageDirectory']
        
        self.Page = str(open(self.userSettingsDict['Page'], "r").read())
        self.Styles = str(open(self.userSettingsDict['Styles'], "r").read())
        self.Java = str(open(self.userSettingsDict['Java'], "r").read())
        
        # Capture Settings
        self.captureStillOn = self.userSettingsDict['captureStillOn']
        self.captureDirectory = self.userSettingsDict['captureDirectory']
        self.captureRate = self.userSettingsDict['captureRate']
        
        # Extraction Settings
        self.extractionDirectory = self.userSettingsDict['extractionDirectory']
        self.extractionThreshold = self.userSettingsDict['extractionThreshold']

        # Notification Settings
        self.notificationType = self.userSettingsDict['notificationType']
        self.notDelay = self.userSettingsDict['notDelay']
        self.ownerEmail = self.userSettingsDict['ownerEmail']
     
        # Email Server Settings
        self.emailPort = self.userSettingsDict['emailPort']
        self.smtpServer =  self.userSettingsDict['smtpServer']
        self.senderEmail = self.userSettingsDict['senderEmail']
        
        # might adjust login
        self.notificationAuth = self.userSettingsDict['notificationAuth']

        #Facial Recognition Settings
        self.inferIdentity = self.userSettingsDict['inferIdentity']
        

        self.frameRate = self.userSettingsDict['frameRate']                               #### need to test this ####
        self.resolution = self.userSettingsDict['resolution']                               #### need to test this ####
        self.rotation = self.userSettingsDict['rotation']                               #### need to test this ####
        self.turingCoreTFLITE = self.userSettingsDict['turingCoreTFLITE']                               #### need to test this ####
        self.idCoreTFLITE = self.userSettingsDict['idCoreTFLITE']                               #### need to test this ####
        self.boltControl = self.userSettingsDict['boltControl'] #### need to test this ####
        self.boltDelay = self.userSettingsDict['boltDelay']
        
        self.networkAddress = self.userSettingsDict['networkAddress']                               #### need to test this ####
        self.networkPort = self.userSettingsDict['networkPort']                               #### need to test this ####
        self.idThres = self.userSettingsDict['idThres']                               #### need to test this ####
        self.turingThres = self.userSettingsDict['turingThres'] 
        # still need figured out ####
        self.authGuest = []
        for i in range(0, len(self.authorizationsDict)):
            self.authGuest.append(self.authorizationsDict[i])
            
        self.shutdownFlag = 'R'

    def saveSettings(self):
        # provide implementation for saving settings to file
        with open(r'setup/settings.yaml', 'w') as file:
            yaml.dump(self.userSettingsDict, file)
            
        authorizationsDict = { i : self.authGuest[i] for i in range(0, len(self.authGuest) ) }
        with open(r'setup/authorizations.yaml', 'w') as file:
            yaml.dump(authorizationsDict, file)

    def settingDisplay(self):
        return self.userSettingsDict


    # The Setters
    def setStorageDirectory(self, passedValue):
        self.storageDirectory = passedValue
        self.userSettingsDict['storageDirectory'] = passedValue
        return

    def setPage(self, passedValue):
        self.Page = passedValue
        self.userSettingsDict['Page'] = passedValue

    def setStyles(self, passedValue):
        self.Styles = passedValue
        self.userSettingsDict['Styles'] = passedValue
        return

    def setJava(self, passedValue):
        self.Java = passedValue
        self.userSettingsDict['Java'] = passedValue
        return

    def setCaptureStillOn(self, passedValue):
        self.captureStillOn = passedValue
        self.userSettingsDict['captureStillOn'] = passedValue
        return

    def setCaptureDirectory(self, passedValue):
        self.captureDirectory = passedValue
        self.userSettingsDict['captureDirectory'] = passedValue
        return
 
    def setCaptureRate(self, passedValue):
        self.captureRate = passedValue
        self.userSettingsDict['captureRate'] = passedValue
        return

    def setExtractionDirectory(self, passedValue):
        self.extractionDirectory = passedValue
        self.userSettingsDict['extractionDirectory'] = passedValue
        return

    def setExtractionThreshold(self, passedValue):
        self.extractionThreshold = passedValue
        self.userSettingsDict['extractionThreshold'] = passedValue
        return

    def setNotificationType(self, passedValue):
        self.notificationType = passedValue
        self.userSettingsDict['notificationType'] = passedValue
        return

    def setJava(self, passedValue):
        self.notDelay = passedValue
        self.userSettingsDict['notDelay'] = passedValue
        return

    def setOwnerEmail(self, passedValue):
        self.ownerEmail = passedValue
        self.userSettingsDict['ownerEmail'] = passedValue
        return

    def setEmailPort(self, passedValue):
        self.emailPort = passedValue
        self.userSettingsDict['emailPort'] = passedValue  
        return

    def setSmtpServer(self, passedValue):
        self.smtpServer = passedValue
        self.userSettingsDict['smtpServer'] = passedValue
        return

    def setSenderEmail(self, passedValue):
        self.senderEmail = passedValue
        self.userSettingsDict['senderEmail'] = passedValue
        return

    def setNotificationAuth(self, passedValue):
        self.notificationAuth = passedValue
        self.userSettingsDict['notificationAuth'] = passedValue
        return

    def setInferIdentity(self, passedValue):
        self.inferIdentity = passedValue
        self.userSettingsDict['inferIdentity'] = passedValue   
        return
    
    def setBoltControl(self, passedValue):
        self.boltControl = passedValue
        self.userSettingsDict['boltControl'] = passedValue
        return
    
    def setRotation(self, passedValue):
        self.rotation = passedValue
        self.userSettingsDict['rotation'] = passedValue
        return
    
    def setShutdownFlag(self, passedValue):
        self.shutdownFlag = passedValue
        self.userSettingsDict['shutdownFlag'] = passedValue        
        return
    



    # The Getters
    def getStorageDirectory(self):
        return self.storageDirectory

    def getPage(self):
        return self.Page

    def getStyles(self):
        return self.Styles

    def getJava(self):
        return self.Java

    def getCaptureStillOn(self):
        return self.captureStillOn

    def getCaptureDirectory(self):
        return self.captureDirectory

    def getCaptureRate(self):
        return self.captureRate

    def getExtractionDirectory(self):
        return self.extractionDirectory

    def getExtractionThreshold(self):
        return self.extractionThreshold

    def getNotificationType(self):
        return self.notificationType

    def getJava(self):
        return self.notDelay

    def getOwnerEmail(self):
        return self.ownerEmail

    def getEmailPort(self):
        return self.emailPort

    def getSmtpServer(self):
        return self.smtpServer

    def getSenderEmail(self):
        return self.senderEmail

    def getNotificationAuth(self):
        return self.notificationAuth

    def getInferIdentity(self):
        return self.inferIdentity

    def getAuthGuest(self, i):
        return self.authGuest[i]

    def getBoltControl(self):
        return self.boltControl

    def getRotation(self):
        return self.rotation
    
    def getShutdownFlag(self):
        return self.shutdownFlag

    

    def handleInput(self, fields):
        print("Fields:", fields)
        test = str(fields.get('notAdd'))
        print("FieldsValue: ", test)
        
        if test=="['michael.c.locker@gmail.com']":
            self.setOwnerEmail('michael.c.locker@gmail.com')
            print("Notification Email Changed to: 'michael.c.locker@gmail.com' ")
            
        elif test=="['jamedaful@gmail.com']":
            self.setOwnerEmail('jamedaful@gmail.com')
            print("Notification Email Changed to: 'jamedaful@gmail.com' ")
            
        elif test=="['notify.lockindustries@gmail.com']":
            self.setOwnerEmail('notify.lockindustries@gmail.com')
            print("Notification Email Changed to: 'notify.lockindustries@gmail.com' ")
            
        elif test=="['NotificationMotion']":
            self.setNotificationType(1)
            print("Notifications are now set to motion. ")
            
        elif test=="['NotificationTuring']":
            self.setNotificationType(2)
            print("Notifications are now set to turing. ")
            
        elif test=="['NotificationOff']":
            self.setNotificationType(0)
            print("Notifications are now turned off. ")
            
        elif test=="['CaptureOn']":
            self.setCaptureStillOn(True)
            print("Still Capturing is now turned on. ")
            
        elif test=="['CaptureOff']":
            self.setCaptureStillOn(False)
            print("Still Capturing is now turned off. ")
            
        elif test=="['rotation90']":
            self.setRotation(False)
            print("Rotation is now set to 90. ")
            
        elif test=="['rotation0']":
            self.setRotation(False)
            print("Rotation is now set to 0. ")
            
        elif test=="['rotation180']":
            self.setRotation(False)
            print("Rotation is now set to 180. ")
            
        elif test=="['rotation270']":
            self.setRotation(False)
            print("Rotation is now set to 270. ")

        elif test=="['boltControlOn']":
            self.setBoltControl(True)
            print("Bolt Control is now turned on. ")
            
        elif test=="['boltControlOff']":
            self.setBoltControl(False)
            print("Bolt Control is now turned off. ")
        
        elif test=="['SaveSettings']":
            print("Saving Settings . . . ")
            self.saveSettings()
            print("Saved!")
            
        elif test=="['ExtractImages']":
            moCap.extractFiles(self.storageDirectory, self.captureDirectory, self.extractionDirectory)
            moCap.imgNum = 0
            print("Extracting Still Images. ")
            
        elif test=="['ServerShutdown']":
            self.setShutdownFlag('x')
            print("Server will shutdown")
            
        else:
            print("Invalid Input")
            
        return 

In [7]:
#!/usr/bin/env python3
import socketserver, picamera, io, logging
from http import server
from threading import Condition
from urllib.parse import parse_qs

## Create Settings Manager  (Must instantiate too intialize from file first)
userSettings = settingManager()

## Create Notification Manager
noteMan = notificationManager()

## Create Motion Detector
moCap = motionCapture()   

## Create Network Core
turingCore = modelFrame(userSettings.turingCoreTFLITE,userSettings.turingThres)
idCore = modelFrame(userSettings.idCoreTFLITE,userSettings.idThres)
# Call is: 
# result1 = turingCore.inference(sample)
# result2 = idCore.inference(sample)

## Begin Streaming
PAGE = userSettings.getPage()
STYLESHEET = userSettings.getStyles()
JAVASCRIPT =  userSettings.getJava()

class StreamingOutput(object):

    def __init__(self):
        self.frame = None
        self.buffer = io.BytesIO()
        self.condition = Condition()

    def write(self, buf):
        if buf.startswith(b'\xff\xd8'):
            # New frame, copy the existing buffer's content and notify all
            # clients it's available
            self.buffer.truncate()
            with self.condition:
                self.frame = self.buffer.getvalue()
                self.condition.notify_all()
                self.buffer.seek(0)
                moCap.processFrame(self.frame)

        return self.buffer.write(buf)

    
    
class StreamingHandler(server.BaseHTTPRequestHandler):
    
    def _set_headers(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
    
    def do_POST(self):
        print(self.headers)

        content_length = int(self.headers.get('Content-Length', 0))
        content_string = self.rfile.read(content_length).decode("UTF-8")
        fields = parse_qs(content_string)
        userSettings.handleInput(fields)
        
        self.send_response(302)
        self.send_header('Location', '/index.html')
        self.end_headers()
        self._set_headers()   
        return
    
    def do_GET(self):
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/index.html')
            self.end_headers()
        elif self.path == '/index.html':
            content = PAGE.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/html')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/style.css':
            content = STYLESHEET.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/css')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/javascript.js':
            content = JAVASCRIPT.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/javascript')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/stream.mjpg':
            self.send_response(200)
            self.send_header('Age', 0)
            self.send_header('Cache-Control', 'no-cache, private')
            self.send_header('Pragma', 'no-cache')
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=FRAME')
            self.end_headers()

            try:
                while True:
                    with output.condition:
                        output.condition.wait()
                        frame = output.frame

                        self.wfile.write(b'--FRAME\r\n')
                        self.send_header('Content-Type', 'image/jpeg')
                        self.send_header('Content-Length', len(frame))
                        self.end_headers()
                        self.wfile.write(frame)
                        self.wfile.write(b'\r\n')

            except Exception as e:
                if userSettings.getShutdownFlag()=="x":
                    camera.close()
                    server.shutdown()
                    server.server_close()
                    print("Remote Shutdown Requested.")
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()

            
class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True
    

with picamera.PiCamera(resolution=userSettings.resolution, framerate=userSettings.frameRate) as camera:
    camera.rotation = userSettings.rotation
    output = StreamingOutput()
    camera.start_recording(output, format='mjpeg')

    try:
        address = (userSettings.networkAddress, userSettings.networkPort)
        server = StreamingServer(address, StreamingHandler)
        print("Starting Server . . .")
        if userSettings.getNotificationType()==0:
            print("Notifications are disabled.")
        elif userSettings.getNotificationType()==1:
            print("Notifications are set to Motion.")
        elif userSettings.getNotificationType()==2:
            print("Notifications are set to Turing.")
        server.serve_forever() #poll_interval=0.5)

    except KeyboardInterrupt:
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.
        print("Shutdown requested")

    except Exception:
        print("Encountered Unhandled Error!")
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.

    finally:
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.

Starting Server . . .
Notifications are disabled.


10.0.0.163 - - [30/Oct/2020 17:48:15] "GET /index.html HTTP/1.1" 200 -
10.0.0.163 - - [30/Oct/2020 17:48:15] "GET /style.css HTTP/1.1" 200 -
10.0.0.163 - - [30/Oct/2020 17:48:15] "GET /stream.mjpg HTTP/1.1" 200 -
10.0.0.163 - - [30/Oct/2020 17:48:15] code 404, message Not Found
10.0.0.163 - - [30/Oct/2020 17:48:15] "GET /favicon.ico HTTP/1.1" 404 -




Loading . . .
Core Loaded: [{'name': 'Identity', 'index': 188, 'shape': array([1, 8]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}}]
8 neural outputs, and 8 in authorization List 
Correlating Multiclass Model containing 8 classes.


Loading . . .
Core Loaded: [{'name': 'Identity', 'index': 188, 'shape': array([1, 8]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}}]
8 neural outputs, and 8 in authorization List 
Correlating Multiclass Model containing 8 classes.

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Motion Detected at 103020_224817
-----------------------------------
Running Turing Check . . . . . . . 
 . . . . . . . . Humanity Validated
-----------------------------------
Running Identification . . . .

#### Currently you can monitor the feed at this address:

http://10.0.0.30:7777
